In [2]:
from eth_utils import address
from web3 import Web3
import os
from solcx import compile_standard, install_solc
import json

In [14]:
version = '0.8.0'
install_solc(version)

<Version('0.8.0')>

In [15]:
smart_contract = "EnergyTrading.sol"

with open(smart_contract, "r") as file:
    contract_file = file.read()

# Step 1: Se conneceter au réseau (Protocole RPC)
# Step 2: Ajouter son wallet
# Step 3: Déclarer l'adresse et l'ABI
# Step 4:
## Step 4.1: Get data
## Step 4.2:
### Step 4.2.1: Créer une transaction
### Step 4.2.2: Signer la transaction
### Step 4.2.3: Upload la transaction au réseau

In [16]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"EnergyTrading.sol": {"content": contract_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version=version,  # type: ignore
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [17]:
# get bytecode
bytecode = compiled_sol["contracts"][smart_contract]["EnergyTrading"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(compiled_sol["contracts"][smart_contract]["EnergyTrading"]["metadata"])["output"]["abi"]

In [18]:
# set up connection
w3 = Web3(Web3.HTTPProvider("HTTP://172.21.128.1:7545"))
chain_id = 1337
owner_address = "0xD337bc9e419282Cc22E52CeccF8375a651e59711"
private_key = "4c251d2e936cf81982e203b6d630bd7b22ed05f60fffbe3fbcfa440f9370827e"

In [19]:
# initialize contract
EnergyTrading = w3.eth.contract(abi=abi, bytecode=bytecode)
nonce = w3.eth.get_transaction_count(owner_address)

In [ ]:
# set up transaction from constructor which executes when firstly
transaction = EnergyTrading.constructor().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": owner_address, 
        "nonce": nonce
    }
)

signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# dump contract address and abi to files
with open("contract_address.txt", "w") as file:
    file.write(tx_receipt.contract_address)

with open("contract_abi.json", "w") as file:
    json.dump(abi, file)

In [37]:
with open("contract_abi.json", "w") as file:
    json.dump(abi, file)

In [33]:
energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xA7e37689F9C63dB93291EE13FD7666E2C8Cc0B49"
private_key = "8417779402eae454d51d3633bf9f649a0d9f64f553400f61680e5900efcbc8fb"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.registerUser().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

ContractLogicError: ('execution reverted: VM Exception while processing transaction: revert User already registered.', {'stack': 'RuntimeError: VM Exception while processing transaction: revert User already registered.\n    at Function.RuntimeError.fromResults (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\lib\\utils\\runtimeerror.js:94:13)\n    at module.exports (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\lib\\utils\\gas\\guestimation.js:142:32)', 'name': 'RuntimeError'})

In [ ]:
w3.eth.get_balance(tx_receipt.contractAddress) / 1e18